In [47]:
from pathlib import Path

import polars as pl

from stocksense.database_handler import DatabaseHandler

DATE = "2023-06-01"
REPORT_DIR = Path("../reports/scores")

In [48]:
stock_info = DatabaseHandler().fetch_stock()
df = pl.read_csv(REPORT_DIR / f"scores_{DATE}.csv")
df = df.join(stock_info, on="tic", how="left")
df

2024-12-05 17:05:11.385 | SUCCESS  | stocksense.database_handler.schema:create_tables:121 - Tables created successfully


tic,adj_close,freturn,excess_return,fsharpe_ratio,pred,name,sector,date_added,date_removed
str,f64,f64,f64,f64,f64,str,str,date,date
"""NCLH""",15.51,7.027725,-18.001527,-5.545556,37.17069,"""Norwegian Cruise Line Holdings""","""Consumer Discretionary""",2017-10-13,null
"""ZION""",26.068811,63.006534,37.977282,13.678461,30.592213,"""Zions Bancorporation""","""Financials""",null,2024-03-18
"""META""",271.791656,71.426327,46.397075,20.953845,28.327505,"""Meta Platforms""","""Communication Services""",2013-12-23,null
"""MSFT""",328.746765,25.797535,0.768283,0.584343,21.75202,"""Microsoft""","""Information Technology""",1994-06-01,null
"""NVDA""",39.750584,175.756012,150.726759,54.877978,21.142149,"""Nvidia""","""Information Technology""",2001-11-30,null
…,…,…,…,…,…,…,…,…,…
"""MMM""",73.902962,34.062467,9.033215,4.93493,-17.176619,"""3M""","""Industrials""",1957-03-04,null
"""HSY""",249.372787,-21.853062,-46.882314,-37.035982,-18.022165,"""Hershey's""","""Consumer Staples""",1957-03-04,null
"""EBAY""",41.939377,28.145908,3.116655,1.841128,-19.202547,"""eBay""","""Consumer Discretionary""",2002-07-22,null


In [49]:
n = 20
top = df.head(n)
bottom = df.tail(n)

# Calculate average returns for top stocks
top_freturn = top.select(pl.col("excess_return")).mean().item()

# Calculate average returns for bottom stocks
bottom_freturn = bottom.select(pl.col("excess_return")).mean().item()

print(f"\nDATE {DATE}")
print(f"\nTop {n} stocks:")
print(f"Average freturn: {top_freturn:.2f}%")
print(f"\nBottom {n} stocks:")
print(f"Average freturn: {bottom_freturn:.2f}%")


DATE 2023-06-01

Top 20 stocks:
Average freturn: 16.52%

Bottom 20 stocks:
Average freturn: -9.87%


In [14]:
top

tic,close,freturn,excess_return,pred,name,sector,date_added,date_removed
str,f64,f64,f64,f64,str,str,date,date
"""CAH""",55.43,55.022265,52.101433,37.89168,"""Cardinal Health""","""Health Care""",1997-05-27,null
"""LYV""",94.510002,-14.157237,-17.078069,20.778915,"""Live Nation Entertainment""","""Communication Services""",2019-12-23,null
"""MRNA""",143.399994,-10.376562,-13.297394,20.692898,"""Moderna""","""Health Care""",2021-07-21,null
"""PEP""",166.490005,13.162956,10.242124,18.175556,"""PepsiCo""","""Consumer Staples""",1957-03-04,null
"""PGR""",118.470001,8.662078,5.741246,17.985939,"""Progressive Corporation""","""Financials""",1997-08-04,null
…,…,…,…,…,…,…,…,…
"""BA""",130.649994,59.173377,56.252544,13.225108,"""Boeing""","""Industrials""",1957-03-04,null
"""GILD""",64.769997,22.639925,19.719093,13.121877,"""Gilead Sciences""","""Health Care""",2004-07-01,null
"""AAPL""",148.710007,21.824561,18.903729,13.068698,"""Apple Inc.""","""Information Technology""",1982-11-30,null
